In [1]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=16, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-10-07 19:49:53.600 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\mwien\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:00<00:00, 3496.19it/s]


In [3]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))

In [4]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [5]:
loss_fn = torch.nn.CrossEntropyLoss()

In [6]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [7]:
settings = TrainerSettings(
    epochs=16, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": True, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 16
metrics: [Accuracy]
logdir: gestures
train_steps: 162
valid_steps: 40
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': True, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [8]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [ ]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)
    
n_repeats = 3

for repeat in range(n_repeats):
    print(f"Repeat {repeat+1}/{n_repeats}")
    
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", f"{repeat}_GRU_16epochs_0.5drop_128hidsize_16batch_7numlayer")
        mlflow.set_tag("dev", "Marcello")
        # Log hyperparameters to MLflow

        mlflow.log_param("epochs", settings.epochs)

        mlflow.log_param("learning_rate", settings.optimizer_kwargs.get("lr", None))
        
        config = ModelConfig(
            input_size=3,
            hidden_size=128,
            num_layers=7,
            output_size=20,
            dropout=0.5,
        )

        model = GRUmodel(
            config=config,
        )

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        if not settings.earlystop_kwargs["save"]:
            tag = datetime.now().strftime("%Y%m%d-%H%M-")
            modelpath = modeldir / (tag + "model.pt")
            torch.save(model, modelpath)

2025/10/07 19:49:55 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/07 19:49:55 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


Repeat 1/3


2025-10-07 19:49:55.919 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251007-194955
2025-10-07 19:49:57.067 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 162/162 [00:12<00:00, 12.69it/s]
2025-10-07 19:50:10.666 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.4452 test 1.9168 metric ['0.2906']
2025-10-07 19:50:10.667 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.9168 --> 1.9168).Saving gestures\20251007-194955\checkpoint.pt ...
100%|██████████| 162/162 [00:13<00:00, 12.09it/s]
2025-10-07 19:50:24.846 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.6203 test 1.2620 metric ['0.4609']
2025-10-07 19:50:24.847 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.9168 --> 1.2620).Saving gestures\20251007-194955\checkpoint.pt ...
100%|██████████| 162/162 [00:13<00:00, 12.17it/s]
2025-10-07 19:50:38.903 

Repeat 2/3


100%|██████████| 162/162 [00:17<00:00,  9.12it/s]
2025-10-07 19:54:37.522 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.4688 test 2.1178 metric ['0.1891']
2025-10-07 19:54:37.525 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.1178 --> 2.1178).Saving gestures\20251007-195418\checkpoint.pt ...
100%|██████████| 162/162 [00:15<00:00, 10.80it/s]
2025-10-07 19:54:53.370 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.7909 test 1.5110 metric ['0.3141']
2025-10-07 19:54:53.371 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.1178 --> 1.5110).Saving gestures\20251007-195418\checkpoint.pt ...
100%|██████████| 162/162 [00:15<00:00, 10.55it/s]
2025-10-07 19:55:09.736 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.5306 test 1.3865 metric ['0.3797']
2025-10-07 19:55:09.737 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.5110 --> 1.3865).Saving gestures\20251007-195418\checkpoint.pt ...
100%

In [ ]:
mlflow.end_run()

## Open MLFlow
"""
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
        
mlflow server --backend-store-uri sqlite:///3-hypertuning-rnn/mlflow.db --host 127.0.0.1 --port 5000
```
"""